In [1]:
import src.utils.utils as utils
from src.semantic_parser.learn_framework import EncoderDecoderLFramework
from src.semantic_parser.ensemble_configs import model_dirs as ensemble_model_dirs
from src.eval.wikisql.lib.dbengine import DBEngine
import src.eval.eval_tools as eval_tools
from src.demos.demos import Text2SQLWrapper
from src.data_processor.path_utils import get_model_dir, get_checkpoint_path
from src.data_processor.schema_graph import SchemaGraph
from src.data_processor.vocab_processor import build_vocab
from src.data_processor.data_processor import preprocess
import src.data_processor.processor_utils as data_utils
import src.data_processor.data_loader as data_loader
import src.common.ops as ops
import random
import json
import os
import sys
import torch

In [2]:
from src.parse_args import args

In [3]:
args.data_dir="data/ko_token"
args.db_dir="data/ko_token"
args.dataset_name="wikisql"
args.model="bridge"
args.model_id = 2
args.question_split=True
args.query_split=False
args.question_only=True
args.normalize_variables=False
args.denormalize_sql=True
args.omit_from_clause=True
args.no_join_condition=False
args.table_shuffling=True
args.use_graph_encoding=False
args.use_typed_field_markers=False
args.use_lstm_encoder=True
args.use_meta_data_encoding=True
args.use_picklist=True
args.no_anchor_text=False
args.anchor_text_match_threshold=0.85
args.top_k_picklist_matches=2
args.atomic_value_copy=False
args.process_sql_in_execution_order=False
args.sql_consistency_check=False
args.share_vocab=False
args.sample_ground_truth=False
args.save_nn_weights_for_visualizations=True
args.vocab_min_freq=0
args.text_vocab_min_freq=0
args.program_vocab_min_freq=0
args.max_in_seq_len=512
args.max_out_seq_len=60

args.num_steps=10000
args.curriculum_interval=0
args.num_peek_steps=400
args.num_accumulation_steps=3
args.save_best_model_only=True
args.train_batch_size=8
args.dev_batch_size=8
args.encoder_input_dim=768
args.encoder_hidden_dim=512
args.decoder_input_dim=512
args.num_rnn_layers=1
args.num_const_attn_layers=0
args.use_oracle_tables=False
args.num_random_tables_added=0
args.use_additive_features=False
args.schema_augmentation_factor=1
args.random_field_order=False
args.data_augmentation_factor=1
args.augment_with_wikisql=False
args.num_values_per_field=0
args.pretrained_transformer="bert-base-multilingual-cased"
args.fix_pretrained_transformer_parameters=False
args.bert_finetune_rate=0.00005
args.learning_rate=0.0003
args.learning_rate_scheduler="inverse-square"
args.trans_learning_rate_scheduler="inverse-square"
args.warmup_init_lr=0.0003
args.warmup_init_ft_lr=0
args.num_warmup_steps=3000
args.emb_dropout_rate=0.3
args.pretrained_lm_dropout_rate=0
args.rnn_layer_dropout_rate=0.1
args.rnn_weight_dropout_rate=0
args.cross_attn_dropout_rate=0
args.cross_attn_num_heads=8
args.res_input_dropout_rate=0.2
args.res_layer_dropout_rate=0
args.ff_input_dropout_rate=0.4
args.ff_hidden_dropout_rate=0.0
args.grad_norm=0.3
args.decoding_algorithm="beam-search"
args.beam_size=8
args.bs_alpha=1.0
args.data_parallel=False

args.model_dir = './model/wikisql.bridge.lstm.meta.ts.ko_token.bs_8.ppl-0.85.2.dn.no_from.feat.bert-base-multilingual-cased.xavier-768-512-512-8-3-0.0003-inv-sqr-0.0003-3000-5e-05-inv-sqr-0.0-3000-0.3-0.3-0.0-0.0-1-8-0.1-0.0-res-0.2-0.0-ff-0.4-0.0.test'

In [4]:
sp = EncoderDecoderLFramework(args).cuda()

sp.load_checkpoint(get_checkpoint_path(args))
sp.eval()

* text vocab size = 119547
* program vocab size = 99

pretrained_transformer = bert-base-multilingual-cased
fix_pretrained_transformer_parameters = False

share_vocab is False
bridge module created


/home/hoonst/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


=> loading checkpoint './model/wikisql.bridge.lstm.meta.ts.ko_token.bs_8.ppl-0.85.2.dn.no_from.feat.bert-base-multilingual-cased.xavier-768-512-512-8-3-0.0003-inv-sqr-0.0003-3000-5e-05-inv-sqr-0.0-3000-0.3-0.3-0.0-0.0-1-8-0.1-0.0-res-0.2-0.0-ff-0.4-0.0.test/model-best.8.tar'


EncoderDecoderLFramework(
  (mdl): Bridge(
    (encoder_embeddings): TransformerHiddens(
      (trans_parameters): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelf

In [5]:
dataset = data_loader.load_processed_data(args)

split = 'test'
if args.dataset_name == 'wikisql':
    engine_path = os.path.join(args.data_dir, '{}.db'.format(split))
    engine = DBEngine(engine_path)
else:
    engine = None

loading preprocessed data: data/ko_token/wikisql.bridge.question-split.ppl-0.85.2.dn.no_from.bert.multilingual.ko_token.pkl


In [6]:
dataset['test'][0].text = 'terrence ross가 뭐하는 새끼야 '

In [7]:
print(f'db_id: {dataset["test"][0].db_id}')
print(f'db_name: {dataset["test"][0].db_name}')

db_id: 21301
db_name: 1-10015132-16


In [8]:
examples = dataset[split][0:1]
sp.schema_graphs = dataset['schema']

In [9]:
print('{} {} examples loaded'.format(len(examples), split))

1 test examples loaded


In [10]:
pred_restored_cache = sp.load_pred_restored_cache()
pred_restored_cache_size = sum(len(v) for v in pred_restored_cache.values())

out_dict = sp.inference(examples, restore_clause_order=args.process_sql_in_execution_order,
                            pred_restored_cache=pred_restored_cache,
                            check_schema_consistency_=args.sql_consistency_check,
                            engine=engine, inline_eval=True, verbose=True)

100%|██████████| 1/1 [00:00<00:00,  3.90it/s]


In [11]:
from src.data_processor.processors.data_processor_wikisql import generate_sql_q, generate_sql_q1

In [12]:
pred_sql = [{'sql': {'sel': 2, 'conds': [[0, 0, 'Terrence Ross']], 'agg': 0}, 'table_id': '1-10015132-16'}]
pred_sql_table_id = pred_sql[0]['table_id']
pred_sqls = [pred_sql[0]['sql']]

In [13]:
import json
import ast

with open('./data/ko_from_table/test.tables.jsonl', 'r') as json_file:
    tables = list(json_file)

pred_sql = [{'sql': {'sel': 2, 'conds': [[0, 0, 'Terrence Ross']], 'agg': 0}, 'table_id': '1-10015132-16'}]
pred_sql_table_id = pred_sql[0]['table_id']
pred_sqls = [pred_sql[0]['sql']]
# pred_sql_table_id = 'table_' + pred_sql[0]['table_id'].replace('-', '_')

for i, table in enumerate(tables):
    tables[i] = ast.literal_eval(table)

In [14]:
target_table = [table for table in tables if table["id"] == pred_sql_table_id]

In [15]:
generate_sql_q(pred_sqls, target_table)

['SELECT (Nationality) FROM 1-10015132-16 WHERE Player = Terrence Ross']

In [16]:
engine.execute('1-10015132-16', 2, 0, [[0, 0, 'Terrence Ross']])

Parsed Query SELECT col2 AS result FROM table_1_10015132_16 WHERE col0 = :col0
Executed Output {out}


['united states']